In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice
from collections import Counter, defaultdict

from itertools import chain
import ast


import statistics
import numpy as np

In [2]:
# Importation données
# toEval = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/toEval/2016_G06F_patents_toEval.csv') # /home/edgarlanoue/data/csv/toEval/2016_G06F_patents_toEval.csv 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/toEval/2016_G06F_patents_toEval.csv'
# KS = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/KS/2016_1115_G06F_KS_raw.csv') # /home/edgarlanoue/data/csv/KS/2016_1115_G06F_KS_raw.csv C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/KS/2016_1115_G06F_KS_raw.csv
toEval = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/PatentNovelty/pickledDF/2016_G06F_patents_toEval.pkl')
KS = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/PatentNovelty/pickledDF/2016_1115_G06F_KS_raw.pkl')
print("toEval lenght: ", len(toEval))
print("KS lenght: ", len(KS))



# Création différents jdD
toEval_ipc = list([ast.literal_eval(i) for i in toEval.sec_ipc])
KS_ipc = list(set(chain.from_iterable([ast.literal_eval(s) for s in KS.sec_ipc])))
KS_sec_ipc = list(pd.Series([ast.literal_eval(i) for i in KS.sec_ipc]))
# print(toEval_ipc)
# print(KS_ipc)
# print("toEval length ipc: ", len(toEval_ipc))
# print("KS length of unique ipc: ", len(KS_ipc))



# set_toEval_ipc = [set(sublist) for sublist in toEval_ipc]
# set_KS_ipc = [set(sublist) for sublist in KS_sec_ipc]



toEval lenght:  9234
KS lenght:  173662


### Newness

In [ ]:
from novelty_ipc import Newness

instanceNewness = Newness(known_ipc=KS_ipc, list_new_Q=toEval_ipc)

In [19]:
scoreNewness = instanceNewness.novelty_score_norm()

100%|██████████| 9234/9234 [00:01<00:00, 6333.79it/s]


In [20]:
scoreNewness[1]

0.042343513103747024

In [46]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)

<module 'novelty_ipc' from 'c:\\Users\\edgar\\OneDrive\\Bureau\\Ecole\\HEC\\A24\\BrevetNLP\\PatentNovelty\\ipcMetrics\\novelty_ipc.py'>

### Uniqueness

In [3]:
from novelty_ipc import Uniqueness

instanceUnique = Uniqueness(list_new_Q=toEval_ipc, list_known_P=KS_sec_ipc)

In [5]:
Counter(instanceUnique.dist_to_proto())

Counter({1.0: 7651,
         0.0: 595,
         0.5: 455,
         0.6666666666666667: 275,
         0.75: 139,
         0.8: 65,
         0.8333333333333334: 23,
         0.8571428571428572: 16,
         0.8888888888888888: 7,
         0.875: 6,
         0.9230769230769231: 1,
         0.9166666666666666: 1})

### Difference

In [47]:
################################################
#do not forget to take out random state!
#from PatentNovelty.ipcMetrics.novelty_ipc

from novelty_ipc import Difference

instanceDiff10 = Difference(list_new_Q=toEval_ipc, list_known_P=KS_sec_ipc, N=10, nbKS=1000)
# instanceDiff100 = Difference(list_new_Q=KS_sec_ipc, list_known_P=KS_sec_ipc, N=100, nbKS=1000)

In [48]:
print("Seuil avec 10: ", instanceDiff10.neighbor_dist)
# print("Seuil avec 100: ", instanceDiff100.neighbor_dist)

Seuil avec 10:  0.0498352380952381


In [49]:
rTA10 = instanceDiff10.ratio_toAll(n=500)

100%|██████████| 500/500 [01:00<00:00,  8.29it/s]


In [50]:
rTA10_matrix = instanceDiff10.ratio_toAll_matrix(n=500) # petit problème avec la RAM si on veut tout prendre le jdD, sinon plus rapide!

100%|██████████| 500/500 [00:09<00:00, 52.31it/s]


In [52]:
print(np.mean(rTA10))
print(np.mean(rTA10_matrix))

0.988295067429835
0.988295067429835


In [32]:
rTA10_matrix_test = instanceDiff10.ratio_toAll_matrix(n=1000)

100%|██████████| 1000/1000 [00:27<00:00, 36.20it/s]


In [33]:
print(np.mean(rTA10_matrix_test))

0.9904337160691458
